# 13. Modeling (Stata) - all commodities

In [1]:
use "../stata_data/sample_features_2019-09-05.dta"

In [2]:
ds

YEAR          MILES         MILES_1950    MILES_1990    INCIDENTS     AVG_AGE       change_2      miles_avg_3   age_miles     perc_1960     perc_2000     avg_1950      avg_1990      SIGN_INC_M~E  log_sd_cha~e  miles_x_ch~e
COMMODITY     AGE_UNKNOW~S  MILES_1960    MILES_2000    SIGNIFICAN~S  CHANGE        change_3      miles_1950_3  perc_pr~1940  perc_1970     perc_2010     avg_1960      avg_2000      M_A_3         sd_change_sq
OPERATOR_ID   MILES_P~1940  MILES_1970    MILES_2010    GROUP         M_A           mean_chang~3  sign_inc_3    perc_1940     perc_1980     avg_pre_1940  avg_1970      avg_2010      LOG_INCIDE~E  sd_chang~w_3
NAME          MILES_1940    MILES_1980    PARENT        PERC_OFFSH~E  change_1      sd_change_3   inc_per_mi~3  perc_1950     perc_1990     avg_1940      avg_1980      INCIDENTS_~E  LOG_SIGN_I~E  one_over_s~e


In [3]:
egen panelid = group(OPERATOR_ID COMMODITY), label

In [4]:
xtset panelid YEAR

       panel variable:  panelid (unbalanced)
        time variable:  YEAR, 2008 to 2018, but with gaps
                delta:  1 unit


In [6]:
xtreg inc_per_mile_3 sd_change_3 sd_change_sq avg_1940 avg_1950 avg_1960 avg_1970 avg_1980 avg_1990 avg_2000 avg_2010, fe vce(robust)


Fixed-effects (within) regression               Number of obs     =        604
Group variable: panelid                         Number of groups  =        114

R-sq:                                           Obs per group:
     within  = 0.0104                                         min =          1
     between = 0.0003                                         avg =        5.3
     overall = 0.0000                                         max =          6

                                                F(10,113)         =       0.88
corr(u_i, Xb)  = -0.3173                        Prob > F          =     0.5579

                              (Std. Err. adjusted for 114 clusters in panelid)
------------------------------------------------------------------------------
             |               Robust
inc_per_mi~3 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
 sd_change_3 |  -4.24e-06   

In [7]:
local date : di  %tdCY-N-D  daily("$S_DATE", "DMY")

In [8]:
putexcel set "../results/fe_all_`date'.xlsx", replace

Note: file will be replaced when the first putexcel command is issued


In [9]:
matrix D = r(table)'

In [10]:
putexcel A1 = matrix(D), names

file ../results/fe_all_2019-09-06.xlsx saved


## Hausman test

In [11]:
xtreg inc_per_mile_3 sd_change_3 sd_change_sq avg_1940 avg_1950 avg_1960 avg_1970 avg_1980 avg_1990 avg_2000 avg_2010, fe


Fixed-effects (within) regression               Number of obs     =        604
Group variable: panelid                         Number of groups  =        114

R-sq:                                           Obs per group:
     within  = 0.0104                                         min =          1
     between = 0.0003                                         avg =        5.3
     overall = 0.0000                                         max =          6

                                                F(10,480)         =       0.51
corr(u_i, Xb)  = -0.3173                        Prob > F          =     0.8860

------------------------------------------------------------------------------
inc_per_mi~3 |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
 sd_change_3 |  -4.24e-06   4.65e-06    -0.91   0.361    -.0000134    4.88e-06
sd_change_sq |   2.98e-09   4.33e-09     0.69   0.491    -5.52e-

In [12]:
estimate store fixed

In [13]:
xtreg inc_per_mile_3 sd_change_3 sd_change_sq avg_1940 avg_1950 avg_1960 avg_1970 avg_1980 avg_1990 avg_2000 avg_2010, re


Random-effects GLS regression                   Number of obs     =        604
Group variable: panelid                         Number of groups  =        114

R-sq:                                           Obs per group:
     within  = 0.0032                                         min =          1
     between = 0.0502                                         avg =        5.3
     overall = 0.0292                                         max =          6

                                                Wald chi2(10)     =       6.34
corr(u_i, X)   = 0 (assumed)                    Prob > chi2       =     0.7856

------------------------------------------------------------------------------
inc_per_mi~3 |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
 sd_change_3 |  -4.76e-06   4.20e-06    -1.13   0.258     -.000013    3.48e-06
sd_change_sq |   3.42e-09   4.11e-09     0.83   0.405    -4.64e-

In [14]:
hausman fixed ., sigmamore


Note: the rank of the differenced variance matrix (9) does not equal the number of coefficients being tested (10); be sure this is what you expect, or there may be problems computing the test.  Examine the output of your estimators for anything unexpected
        and possibly consider scaling your variables so that the coefficients are on a similar scale.

                 ---- Coefficients ----
             |      (b)          (B)            (b-B)     sqrt(diag(V_b-V_B))
             |     fixed          .          Difference          S.E.
-------------+----------------------------------------------------------------
 sd_change_3 |   -4.24e-06    -4.76e-06        5.15e-07        1.95e-06
sd_change_sq |    2.98e-09     3.42e-09       -4.41e-10        1.31e-09
    avg_1940 |    .0051743     .0018618        .0033124         .005116
    avg_1950 |    .0035782     .0068166       -.0032383        .0059344
    avg_1960 |    .0088428      .002601        .0062418        .0060564
    avg_1970 